In [1]:
import time

def tracer(func): #1                   
    calls = 0          
    print("tracer")
    
    def wrapper(*args, **kwargs): #2
        print("tracer.wrapper")
        nonlocal calls
        calls += 1
        print('call %s to %s' % (calls, func.__name__))
        
        return func(*args, **kwargs) #2
    
    return wrapper #1


def timer(label='', trace=True): #1 parameter
    print("timer")
    
    def decorator(func): #2
        print("timer.decorator")
        
        def wrapper(*args, **kargs): #3
            print("timer.decorator.wrapper")
            start   = time.time()  
            result  = func(*args, **kargs)
            elapsed = time.time() - start
            wrapper.alltime += elapsed
            
            if trace:
                format = '%s%s: %.5f, %.5f'
                values = (label, func.__name__, elapsed, wrapper.alltime)
                print(format % values)
                
            return result #3
        
        wrapper.alltime = 0
        
        return wrapper #2
    
    return decorator #1

## class MetaTrace

In [2]:
from types import FunctionType

class MetaTrace(type): #1
    def __new__(meta, classname, bases, classdict):
        print("MetaTrace.__new__")
        for funcname, func in classdict.items():
            if type(func) is FunctionType:                     
                classdict[funcname] = tracer(func) # =tracer_wrapper               
        return type.__new__(meta, classname, bases, classdict) #1
    
    
class Person(metaclass=MetaTrace):
    def __init__(self, name, pay):
        self.name = name
        self.pay  = pay
        
    def giveRaise(self, percent):
        self.pay *= (1.0 + percent)
    def lastName(self):
        
        return self.name.split()[-1]

bob = Person('Bob Smith', 50000)
sue = Person('Sue Jones', 100000)
print()

print(bob.name, sue.name)
print()

sue.giveRaise(.10)
print('%.2f' % sue.pay)
print()

print(bob.lastName(), sue.lastName())

MetaTrace.__new__
tracer
tracer
tracer
tracer.wrapper
call 1 to __init__
tracer.wrapper
call 2 to __init__

Bob Smith Sue Jones

tracer.wrapper
call 1 to giveRaise
110000.00

tracer.wrapper
call 1 to lastName
tracer.wrapper
call 2 to lastName
Smith Jones


## def decorateAll return class MetaDecorate

In [3]:
def decorateAll(decorator): #1
    print("decoratorAll")
    
    class MetaDecorate(type): #2
        
        # @classmethod
        def __new__(meta, classname, bases, classdict):
            print("decoratorAll.MetaDecorate.__new__")
            
            for funcname, func in classdict.items():
                if type(funcname) is FunctionType:
                    classdict[funcname] = decorator(func) # =trace_wrapper
                     
            return type.__new__(meta, classname, bases, classdict) #2
        
    return MetaDecorate #1

# Person = MetaDecorate(Person.__name__, Person.__bases__, Person.__dict__)
class Person(metaclass=decorateAll(tracer)): 
    def __init__(self, name, pay):
        self.name = name
        self.pay  = pay
        
    def giveRaise(self, percent):
        self.pay *= (1.0 + percent)
        
    def lastName(self):
        return self.name.split()[-1]

bob = Person('Bob Smith', 50000)
sue = Person('Sue Jones', 100000)
print()

print(bob.name, sue.name)
print()

sue.giveRaise(.10)
print('%.2f' % sue.pay)
print()

print(bob.lastName(), sue.lastName())

decoratorAll
decoratorAll.MetaDecorate.__new__

Bob Smith Sue Jones

110000.00

Smith Jones


In [4]:
def decorateAll(decorator): #1
    print("decorateAll")
    
    class MetaDecorate(type): #2
        
        def __new__(meta, classname, bases, classdict):
            print("decorateAll.MetaDecorate.__new__")
            
            for funcname, func in classdict.items():
                if type(funcname) is FunctionType:
                    classdict[funcname] = decorator(func)#=wrapper
                    
            return type.__new__(meta, classname, bases, classdict) #2 create class 
        
    return MetaDecorate #1

#class Person(metaclass=decorateAll(tracer)):
#Person = decorateAll(tracer)(Person.__name__, Person.__bases__, dict(**Person.__dict__),)

#class Person(metaclass=decorateAll(timer())): 
#class Person(metaclass=decorateAll(timer(label='**'))):
,
#class Person(metaclass=decorateAll(timer(label='**')(tracer))):
#class Person(metaclass=decorateAll(tracer(timer(label='**')))):

class Person(metaclass=decorateAll(tracer(timer(label='**')))):
    def __init__(self, name, pay):
        self.name = name
        self.pay  = pay
        
    def giveRaise(self, percent):
        self.pay *= (1.0 + percent)
        
    def lastName(self):
        return self.name.split()[-1]

print('end class')

bob = Person('Bob Smith', 50000)
sue = Person('Sue Jones', 555555)
print()

print(bob.name, sue.name)
print()

sue.giveRaise(.10)
print('%.2f' % sue.pay)
print()

print(bob.lastName(), sue.lastName())

timer
tracer
decorateAll
decorateAll.MetaDecorate.__new__
end class

Bob Smith Sue Jones

611110.50

Smith Jones
